In [4]:
!pip install tensorflow
%load_ext autoreload
%autoreload 2
import glob2
import pandas as pd
import numpy as np
import text_summarization
import extract_text
import sentiment_analyzer
from tqdm import tqdm_pandas
from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import warnings
warnings.filterwarnings('ignore')

  Running setup.py bdist_wheel for absl-py: started
  Running setup.py bdist_wheel for absl-py: finished with status 'done'
  Stored in directory: C:\Users\maxpr\AppData\Local\pip\Cache\wheels\45\07\0e\6880381ca521796cf6cc18ba4ab502c2232e5777099b4df4ae
  Running setup.py bdist_wheel for gast: started
  Running setup.py bdist_wheel for gast: finished with status 'done'
  Stored in directory: C:\Users\maxpr\AppData\Local\pip\Cache\wheels\8e\fa\d6\77dd17d18ea23fd7b860e02623d27c1be451521af40dd4a13e
  Running setup.py bdist_wheel for html5lib: started
  Running setup.py bdist_wheel for html5lib: finished with status 'done'
  Stored in directory: C:\Users\maxpr\AppData\Local\pip\Cache\wheels\6f\85\6c\56b8e1292c6214c4eb73b9dda50f53e8e977bf65989373c962
Successfully built absl-py gast html5lib
  Found existing installation: html5lib 1.0b8
    Uninstalling html5lib-1.0b8:
      Successfully uninstalled html5lib-1.0b8
  Found existing installation: bleach 2.1.2
    Uninstalling bleach-2.1.2:
    

Using TensorFlow backend.


In [22]:
df_files = pd.read_csv("submission_mapper.csv",delimiter="|")
df_files.head()

,id,filename,company
0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel
1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel
2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel
3,3,businessassociatelist--1-.xlsx,Keppel
4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel


In [23]:
df_files['file_path'] = df_files.apply(lambda x : "files/"+x.company+"/"+x.filename,axis=1)
#files.index = files['file_path']
df_files['sentiment'] = np.nan
#files.loc['files/Keppel/02224.doc'].sentiment
df_files.head()

,id,filename,company,file_path,sentiment
0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,NaN
1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,NaN
2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,NaN
3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,NaN
4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,NaN


In [24]:
def find_sentiment(path):
    try:
        text = extract_text.apply(path)
        important_sents = text_summarization.apply(text)
        #print(len(important_sents))
        importance = np.mean(sentiment_analyzer.apply(important_sents))
        return np.mean(importance)
    return -999

In [32]:
from nltk import pos_tag, ne_chunk
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('maxent_ne_chunker')
nltk.download('words')
NER = ne_chunk(pos_tag(word_tokenize("test")))

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\maxpr/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\maxpr/nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


In [33]:
tqdm().pandas(desc="Progress:")
df_files['sentiment'] = df_files.progress_apply(lambda row: find_sentiment(row.file_path), axis=1)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Progress:', max=387), HTML(value='')))

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
1646592/1641221 [==============================] - ETA: 24 - ETA: 16 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 10 - ETA: 10 - ETA: 8 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 3s 2us/step



Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\maxpr\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\maxpr\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\maxpr\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Could not open Excel


KeyboardInterrupt: 

In [9]:
files.head()

,file_path,sentiment
0,files/Keppel/KLL_AR2002a.pdf,NaN
1,files/Keppel/02224.doc,NaN
2,files/Keppel/AB3Q2012.pdf,0.641253
3,files/Keppel/Lewin_130908_Sup.pdf,NaN
4,files/Keppel/AB2-2017.pdf,0.701296


In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11461191964243711662]

In [8]:
sentiment_analyzer.apply(['I am ugly', 'success'])

array([[0.22300728],
       [0.7965621 ]], dtype=float32)